# ZUMI MAIN

In [12]:
from zumi.zumi import Zumi
import time

import csv
from datetime import datetime
import os
from zumi.util.screen import Screen

from zumi.util.camera import Camera
from zumi.util.vision import Vision

from zumi.personality import Personality
from datetime import datetime

ModuleNotFoundError: No module named 'fcntl'

In [3]:
zumi = Zumi()
zumi.reset_gyro()
camera = Camera()
screen = Screen()
personality = Personality(zumi, screen)
vision = Vision()

NameError: name 'Zumi' is not defined

## search_curve

In [4]:
def search_curve(zumi, speed=30, threshold=150):
    zumi.stop()
    print("Linie verloren – starte gezielte Suche...")

    zumi.reset_gyro()
    time.sleep(0.1)

    # === Suche nach Westen/links (90 Grad) ===
    zumi.turn(90)
    time.sleep(0.3)
    ir_readings = zumi.get_all_IR_data()
    right_ir_bottom = ir_readings[1]
    left_ir_bottom = ir_readings[3]

    if left_ir_bottom > threshold and right_ir_bottom > threshold:
        print("Linie im Westen gefunden.")
        zumi.reset_gyro()
        return 0
    if left_ir_bottom > threshold:
        # Versuch Feinjustierung nach links
        print("Linker Sensor sieht Linie – feinjustiere weiter nach links...")
        zumi.turn_left(20)
        time.sleep(0.3)
        ir_readings = zumi.get_all_IR_data()
        if ir_readings[1] > threshold and ir_readings[3] > threshold:
            print("Linie nach Feinjustierung im westen gefunden.")
            zumi.reset_gyro()
            return 0

    # === Suche nach Osten/rechts (-90 Grad) ===
    zumi.turn(-90)  # -90 zurück + 20 zusätzliche Korrekturrichtung
    time.sleep(0.3)
    ir_readings = zumi.get_all_IR_data()
    right_ir_bottom = ir_readings[1]
    left_ir_bottom = ir_readings[3]

    if left_ir_bottom > threshold and right_ir_bottom > threshold:
        print("Linie im Osten gefunden.")
        zumi.reset_gyro()
        return 0
    if right_ir_bottom > threshold:
        # Feinjustierung weiter nach rechts
        print("Rechter Sensor sieht Linie – feinjustiere weiter nach rechts...")
        zumi.turn_right(20)
        time.sleep(0.3)
        ir_readings = zumi.get_all_IR_data()
        if ir_readings[1] > threshold and ir_readings[3] > threshold:
            print("Linie nach Feinjustierung im Osten gefunden.")
            zumi.reset_gyro()
            return 0

    # === Keine Linie gefunden ===
    print("Keine Linie erkannt.")
    return None


## follow_line

In [5]:
def follow_line(zumi, duration=120, speed=30, threshold=150, ir_threshold_left=180, ir_threshold_right=190):
    heading = 0
    start_time = time.time()

    while time.time() - start_time < duration:
        ir_readings = zumi.get_all_IR_data()
        right_ir_bottom = ir_readings[1]
        left_ir_bottom = ir_readings[3]
        left_ir_front = ir_readings[5]
        right_ir_front = ir_readings[0]
        
        # Erste Prüfung – beide Sensoren müssen über der Schwelle sein
        if left_ir_front < ir_threshold_left or right_ir_front < ir_threshold_right:
            object_ahead(zumi, screen, personality, ir_threshold_left, ir_threshold_right)
        

        if left_ir_bottom > threshold and right_ir_bottom <= threshold:
            heading += 5
        elif right_ir_bottom > threshold and left_ir_bottom <= threshold:
            heading -= 5
        elif left_ir_bottom <= threshold and right_ir_bottom <= threshold:
            zumi.stop()
            print("Linie verloren – suche neue Richtung...")
            new_heading = search_curve(zumi, speed, threshold)
            if new_heading is not None:
                heading = new_heading
                continue
            else:
                print("Suche gescheitert – Programmabbruch.")
                break

        zumi.go_straight(speed, heading)
        time.sleep(0.005)

    zumi.stop()


## read_qr_message

In [6]:
def read_qr_message(image):
    qr_code = vision.find_QR_code(image)
    if qr_code is not None:
        message = vision.get_QR_message(qr_code)
        print("QR-Code erkannt:", message)
        return message
    else:
        return None

## handle_qr_action

In [7]:
import re
def handle_qr_action(message, zumi, screen, personality, treshold_white_stop = 100):
    emotions = ["Zumi is happy today!", "Zumi is angry today!", "Zumi is celebrating today!"]
    directions = ["Turn Right", "Turn Left", "Right Circle", "Left Circle"]
    feature_task = ["360 nach Links", "360 nach rechts"]
    if any(keyword in message for keyword in feature_task):
        print("Fahre langsam vor zur Kreuzung...")
        
        match = re.search(r'(\d+)\s*x?\s*360', message)
        if match:
            num_turns = int(match.group(1))  # Extrahierte Zahl (z. B. "2" aus "2x 360 nach links")
        else:
            num_turns = 1  # Falls keine Zahl angegeben ist, gehe von 1 Drehung aus

        while True:
            ir_readings = zumi.get_all_IR_data()
            left_ir_bottom = ir_readings[3]
            right_ir_bottom = ir_readings[1]

            # Schwellenwert z. B. < 100 = weiß erkannt
            if left_ir_bottom <= treshold_white_stop and right_ir_bottom <= treshold_white_stop:
                zumi.stop()
                print("Weiße Fläche erkannt – führe QR-Aktion aus")
                break

            zumi.go_straight(10, 0)  # langsam geradeaus
            time.sleep(0.05)
        if "nach Links" in message and "360" in message:
            print(f"QR-Aktion: {num_turns} mal 360 Grad nach links drehen")
            for _ in range(num_turns):
                zumi.turn_left(360, 3)
            zumi.forward(2)

        elif "nach rechts" in message and "360" in message:
            print(f"QR-Aktion: {num_turns} mal 360 Grad nach rechts drehen")
            for _ in range(num_turns):
                zumi.turn_right(360, 3)
            zumi.forward(2)
    
    elif message in directions:
        print("Fahre langsam vor zur Kreuzung...")
        while True:
            ir_readings = zumi.get_all_IR_data()
            left_ir_bottom = ir_readings[3]
            right_ir_bottom = ir_readings[1]

            # Schwellenwert z. B. < 100 = weiß erkannt
            if left_ir_bottom <= treshold_white_stop and right_ir_bottom <= treshold_white_stop:
                zumi.stop()
                print("Weiße Fläche erkannt – führe QR-Aktion aus")
                break

            zumi.go_straight(10, 0)  # langsam geradeaus
            time.sleep(0.05)
            
              
        if message == "Turn Right":
            print("QR-Aktion: Rechts abbiegen")
            zumi.turn_right(-90)

        elif message == "Turn Left":
            print("QR-Aktion: Links abbiegen")
            zumi.turn_left(90)

        elif message == "Right Circle":
            print("QR-Aktion: Rechtskreis")
            zumi.right_circle(speed=10, step=2)
            zumi.stop()

        elif message == "Left Circle":
            print("QR-Aktion: Linkskreis")
            zumi.left_circle(speed=10, step=2)
            zumi.stop()

        #elif message == "Right Circle":
        #    print("QR-Aktion: Rechtskreis")
        #    for i in range(4):
        #        zumi.turn_right(-90)
        #        zumi.go_straight(30, 0)
        #        time.sleep(1)
        #    zumi.turn_left(90)
        #    zumi.go_straight(30, 0)
        #    time.sleep(1)
        #    zumi.stop()

        #elif message == "Left Circle":
        #    print("QR-Aktion: Linkskreis")
        #    for i in range(4):
        #        zumi.turn_left(90)
        #        zumi.go_straight(30, 0)
        #        time.sleep(1)
        #    zumi.turn_right(-90)
        #    zumi.go_straight(30, 0)
        #    time.sleep(1)
        #    zumi.stop()

    elif message in emotions:
        print("Fahre langsam zur Zielzone...")

        while True:
            ir_readings = zumi.get_all_IR_data()
            left_ir_bottom = ir_readings[3]
            right_ir_bottom = ir_readings[1]

            if left_ir_bottom <= treshold_white_stop and right_ir_bottom <= treshold_white_stop:
                zumi.stop()
                print("Weiße Zielfläche erreicht – führe Emotion aus")
                break

            zumi.go_straight(10, 0)
            time.sleep(0.05)

        if message == "Zumi is happy today!":
            print("QR-Aktion: Freude anzeigen")
            screen.happy()
            personality.happy()

        elif message == "Zumi is angry today!":
            print("QR-Aktion: Wut anzeigen")
            screen.angry()
            personality.angry()

        elif message == "Zumi is celebrating today!":
            print("QR-Aktion: Feier-Modus")
            personality.celebrate()

        print("Zumi ist am Ziel angekommen und bleibt stehen.")
        while True:
            zumi.stop()
            time.sleep(1)  # Endlosschleife = vollständiger Stopp


    #elif message == "Stop":
    #    print("QR-Aktion: Stop – Zumi hält an")
    #    zumi.stop()

    #elif message == "Park":
    #    None

    #elif message == "Drive Round Road":
    #    None


    else:
        print("QR-Inhalt unbekannt:", message)

    time.sleep(1.0)  # kleine Pause nach der Aktion


## object_ahead

In [8]:
def object_ahead(zumi, screen, personality, ir_threshold_left, ir_threshold_right):
    ir_readings = zumi.get_all_IR_data()
    left_ir_front = ir_readings[5]
    right_ir_front = ir_readings[0]

    print("IR vorne (1. Messung): IR0 =", right_ir_front, "IR5 =", left_ir_front)

    # Erste Prüfung – beide Sensoren müssen über der Schwelle sein
    if left_ir_front < ir_threshold_left or right_ir_front < ir_threshold_right:
        zumi.stop()
        print("Objekt bestätigt – Kamera wird aktiviert.")
        time.sleep(0.7)  # kleine Verzögerung

        try:
            camera.start_camera()
            time.sleep(0.9)
            image = camera.capture()
            camera.show_image(image)
            message = read_qr_message(image)
            time.sleep(1.0)

            if message is not None:
                print("QR erkannt – führe Aktion aus")
                handle_qr_action(message, zumi, screen, personality)
                time.sleep(1.0)
            else:
                print("Kein QR-Code – warte auf Objekt-Entfernung...")
                while True:
                    ir_readings = zumi.get_all_IR_data()
                    left_ir_front = ir_readings[5]
                    right_ir_front = ir_readings[0]
                    if left_ir_front > ir_threshold_left and right_ir_front > ir_threshold_right:
                        print("Objekt entfernt – weiterfahren.")
                        break
                    time.sleep(0.2)

        finally:
            camera.close()
            print("Kamera geschlossen.")
        return True

    return False


# Init Func. Full course

In [10]:
def drive_course(zumi, screen, personality, total_time=60, block_duration=5, speed=30, threshold=150, ir_threshold_left=170, ir_threshold_right=180, treshold_white_stop = 100):
    
    print("Starte Parkour-Fahrt mit QR-Logik...")
    start_time = time.time()

    while time.time() - start_time < total_time:
        print("Starte neuen Block...")

        # Linien folgen für festgelegte Zeit
        follow_line(zumi, duration=block_duration, speed=speed, threshold=threshold)

    zumi.stop()
    print("Parkour abgeschlossen.")


In [11]:
# Func zum starten (total time auf contest eingestellt -> 2 mins)
zumi = Zumi()
zumi.mpu.calibrate_MPU()
zumi.reset_gyro()
camera = Camera()
screen = Screen()
personality = Personality(zumi, screen)
vision = Vision()
drive_course(zumi, screen, personality, total_time=120, block_duration=6, speed=15, threshold=150, ir_threshold_left=170, ir_threshold_right=180, treshold_white_stop = 100)

NameError: name 'Zumi' is not defined

In [ ]:
# interrupt Kernel und zumi.stop() wenn verirrt 
zumi.stop()

# TESTS IR, CAM

In [ ]:
ir_readings = zumi.get_all_IR_data()
left_ir_front = ir_readings[5]
right_ir_front = ir_readings[0]

print("IR vorne: IR0 =", right_ir_front, "IR5 =", left_ir_front)

In [ ]:
camera.start_camera()
time.sleep(0.3)
image = camera.capture()
camera.show_image(image)
qr_code = vision.find_QR_code(image)

if qr_code is not None:
    print("QR erkannt:", vision.get_QR_message(qr_code))
else:
    print("Kein QR erkannt.")
camera.close()


# ÄNDERUNGEN

- Besser stoppen mit dynamischen Grenzen von IR Front sensoren nicht fixen schwellenwert -> Tageslichtunabhängig
und/oder Bilderkennung QR codes -> Problem - wann Cam anschalten - müsste durchgehend laufen ? oder in Kombi mit dynamischen IR Grenzen möglich?
- QR Code drive Round Road - alte Methode implementieren oder heading korrektur & time sleep (follow_line) erhöhen


# FAILS:

- IR Front sensoren methode -> Tresholds Lichtabhängig - siehe änderungen oben
- Zumi liest QR code Stop, hält kurz an - fährt dann aber weiter
- Zumi QR Right/Left Circle -> runder Kreis funzt nicht (optional) sonst eif eckiger Kreis fahren - auskommentiert
